# Sentiment Analysis using word2vec
In this tutorial competition, we dig a little "deeper" into sentiment analysis. Google's Word2Vec is a deep-learning inspired method that focuses on the meaning of words. Word2Vec attempts to understand meaning and semantic relationships among words. It works in a way that is similar to deep approaches, such as recurrent neural nets or deep neural nets, but is computationally more efficient. This tutorial focuses on Word2Vec for sentiment analysis.

### Reference
* https://www.kaggle.com/c/word2vec-nlp-tutorial/overview
* https://www.kaggle.com/varun08/sentiment-analysis-using-word2vec/data

In [44]:
# !nltk.download('popular')

In [45]:
# Importing the built-in logging module
import logging

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

In [46]:
# Firstly, please note that the performance of google word2vec is better on big datasets.
# In this example we are considering only 25000 training examples from the imdb dataset.
# Therefore, the performance is similar to the "bag of words" model.

# Importing libraries
import numpy as np
import pandas as pd

# BeautifulSoup is used to remove html tags from the text
from bs4 import BeautifulSoup
import re  # For regular expressions

# Stopwords can be useful to undersand the semantics of the sentence.
# Therefore stopwords are not removed while creating the word2vec model.
# But they will be removed  while averaging feature vectors.
from nltk.corpus import stopwords

# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data


tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

In [47]:
# Read data from files
train = pd.read_csv(
    "./data/labeledTrainData.tsv.gz",
    delimiter="\t",
)
test = pd.read_csv("./data/testData.tsv.gz", delimiter="\t")

In [48]:
# This function converts a text to a sequence of words.
def review_wordlist(review, remove_stopwords=False):
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()

    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]", " ", review_text)

    # 3. Converting to lower case and splitting
    words = review_text.lower().split()

    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return words


# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []

    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_wordlist(raw_sentence, remove_stopwords))

    # This returns the list of lists
    return sentences

In [49]:
# !pip install -U tqdm

In [50]:
from tqdm.auto import tqdm

sentences = []
print("Parsing sentences from training set")
for review in tqdm(train["review"]):
    sentences += review_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\kyoungrok\anaconda3\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kyoungrok\anaconda3\lib\site-packages\bs4\__init__.py:332: MarkupResemblesLocatorWarning: "..." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
C:\Users\kyoungrok\anaconda3\lib\site-packages\bs4\__init__.py:417: MarkupResemblesLocatorWarning: "http://www.happierabroad.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


KeyboardInterrupt: 

In [11]:
import multiprocessing


# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 10  # Minimum word count
num_workers = multiprocessing.cpu_count() / 2  # Number of parallel threads
context = 10  # Context window size
downsampling = 1e-3  # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec

print("Training model....")
model = word2vec.Word2Vec(
    sentences,
    workers=num_workers,
    vector_size=num_features,
    min_count=min_word_count,
    window=context,
    sample=downsampling,
)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "word2vec.model"
model.save(model_name)

C:\Users\kyoungrok\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-07-03 02:54:48,818 : INFO : collecting all words and their counts
2021-07-03 02:54:48,818 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-07-03 02:54:48,881 : INFO : PROGRESS: at sentence #10000, processed 225664 words, keeping 17775 word types
2021-07-03 02:54:48,923 : INFO : PROGRESS: at sentence #20000, processed 451582 words, keeping 24944 word types
2021-07-03 02:54:48,964 : INFO : PROGRESS: at sentence #30000, processed 670632 words, keeping 30023 word types


Training model....


2021-07-03 02:54:49,030 : INFO : PROGRESS: at sentence #40000, processed 896478 words, keeping 34329 word types
2021-07-03 02:54:49,073 : INFO : PROGRESS: at sentence #50000, processed 1115469 words, keeping 37741 word types
2021-07-03 02:54:49,119 : INFO : PROGRESS: at sentence #60000, processed 1336692 words, keeping 40702 word types
2021-07-03 02:54:49,166 : INFO : PROGRESS: at sentence #70000, processed 1559365 words, keeping 43300 word types
2021-07-03 02:54:49,209 : INFO : PROGRESS: at sentence #80000, processed 1778623 words, keeping 45699 word types
2021-07-03 02:54:49,252 : INFO : PROGRESS: at sentence #90000, processed 2002603 words, keeping 48113 word types
2021-07-03 02:54:49,294 : INFO : PROGRESS: at sentence #100000, processed 2224101 words, keeping 50180 word types
2021-07-03 02:54:49,338 : INFO : PROGRESS: at sentence #110000, processed 2442894 words, keeping 52050 word types
2021-07-03 02:54:49,409 : INFO : PROGRESS: at sentence #120000, processed 2665092 words, keepin

KeyboardInterrupt: 

In [27]:
# https://github.com/RaRe-Technologies/gensim-data#models
# It may take some time to finish
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")  # trained with 6B tokens

[====================------------------------------] 40.6% 26.8/66.0MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[====================================--------------] 73.6% 48.6/66.0MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 66.0/66.0MB downloaded


2021-07-04 20:34:15,781 : INFO : glove-wiki-gigaword-50 downloaded
2021-07-04 20:34:15,796 : INFO : loading projection weights from C:\Users\kyoungrok/gensim-data\glove-wiki-gigaword-50\glove-wiki-gigaword-50.gz
2021-07-04 20:34:25,487 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (400000, 50) matrix of type float32 from C:\\Users\\kyoungrok/gensim-data\\glove-wiki-gigaword-50\\glove-wiki-gigaword-50.gz', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-07-04T20:34:25.487455', 'gensim': '4.0.1', 'python': '3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22000-SP0', 'event': 'load_word2vec_format'}


In [16]:
# Few tests: This will print the odd word among them
model.doesnt_match("man woman dog child kitchen".split())

'kitchen'

In [17]:
model.doesnt_match("france england germany berlin".split())

'england'

In [18]:
# This will print the most similar words present in the model
model.most_similar("man")

[('woman', 0.6998663544654846),
 ('person', 0.6443442106246948),
 ('boy', 0.6208277940750122),
 ('he', 0.5926738381385803),
 ('men', 0.5819568634033203),
 ('himself', 0.5810033082962036),
 ('one', 0.5779521465301514),
 ('another', 0.5721587538719177),
 ('who', 0.5703631639480591),
 ('him', 0.5670831203460693)]

In [19]:
model.most_similar("awful")

[('horrible', 0.8252272009849548),
 ('terrible', 0.7882838249206543),
 ('dreadful', 0.7124449610710144),
 ('horrendous', 0.640265703201294),
 ('unbelievable', 0.6266781091690063),
 ('pretty', 0.6222705841064453),
 ('ugly', 0.6063829064369202),
 ('really', 0.6036944389343262),
 ('lousy', 0.5963975787162781),
 ('appalling', 0.5881984233856201)]

### Load the model trained on bigger corpus (for better result)

### Solving Word Analogies!

* Man is to Woman what King is to ___?
* USA is to hamburger what UK is to ___?
* Korea is to kimchi what USA is to ___?

![](./figures/analogy.png)

In [20]:
model.most_similar(positive=["king", "woman"], negative=["man"])
# model.most_similar(positive=["hamburger", "uk"], negative=["usa"])
# model.most_similar(positive=["kimchi", "usa"], negative=["korea"])

[('queen', 0.6713277101516724),
 ('princess', 0.5432624816894531),
 ('throne', 0.5386103987693787),
 ('monarch', 0.5347574949264526),
 ('daughter', 0.49802514910697937),
 ('mother', 0.49564430117607117),
 ('elizabeth', 0.4832652509212494),
 ('kingdom', 0.47747090458869934),
 ('prince', 0.4668239951133728),
 ('wife', 0.46473270654678345)]

In [25]:
# This will give the total number of words in the vocabolary created from this dataset
model.vectors.shape

In [30]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features, index2word_set):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features, dtype="float32")
    nwords = 0

    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model.get_vector(word))

    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")
    # Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.index_to_key)

    for review in tqdm(reviews):
        # Printing a status message every 1000th review
        if counter % 1000 == 0:
            print("Review %d of %d" % (counter, len(reviews)))

        reviewFeatureVecs[counter] = featureVecMethod(
            review, model, num_features, index2word_set
        )
        counter = counter + 1

    return reviewFeatureVecs

In [31]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in tqdm(train["review"]):
    cleaned = review_wordlist(review, remove_stopwords=True)
    clean_train_reviews.append(cleaned)

num_features = model.vectors.shape[1]
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000



In [32]:
# Calculating average feature vactors for test set
clean_test_reviews = []
for review in tqdm(test["review"]):
    cleaned = review_wordlist(review, remove_stopwords=True)
    clean_test_reviews.append(cleaned)

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000



In [33]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100)

print("Fitting random forest to training data....")
forest = forest.fit(trainDataVecs, train["sentiment"])

Fitting random forest to training data....


In [36]:
# Predicting the sentiment values for test data and saving the results in a csv file
result = forest.predict(testDataVecs)
output = pd.DataFrame(data={"id": test["id"], "sentiment": result})
output.to_csv("output.csv", index=False, quoting=3)

Submit the output at https://www.kaggle.com/c/word2vec-nlp-tutorial/leaderboard

# Bonus: Aspect-base Sentiment Analysis 

In [37]:
import spacy
from spacy import displacy
from pprint import pprint

nlp = spacy.load("en_core_web_md")

In [38]:
sentences = [
    "The food we had yesterday was delicious",
    "My time in Italy was very enjoyable",
    "I found the meal to be tasty",
    "The internet was slow.",
    "Our experience was suboptimal",
]

### First, we pick up the sentiment description

In [39]:
for sentence in sentences:
    doc = nlp(sentence)
    descriptive_term = ""
    for token in doc:
        if token.pos_ == "ADJ":
            descriptive_term = token
    print(sentence)
    print(descriptive_term)
    print()

The food we had yesterday was delicious
delicious

My time in Italy was very enjoyable
enjoyable

I found the meal to be tasty
tasty

The internet was slow.
slow

Our experience was suboptimal
suboptimal



### Try to also extract intensifiers (e.g., "very")

In [40]:
for sentence in sentences:
    doc = nlp(sentence)
    descriptive_term = ""
    for token in doc:
        if token.pos_ == "ADJ":
            prepend = ""
            for child in token.children:
                if child.pos_ != "ADV":
                    continue
                prepend += child.text + " "
            descriptive_term = prepend + token.text
    print(sentence)
    print(descriptive_term)
    print()

The food we had yesterday was delicious
delicious

My time in Italy was very enjoyable
very enjoyable

I found the meal to be tasty
tasty

The internet was slow.
slow

Our experience was suboptimal
suboptimal



### Now, identify the targets of the sentiments

In [41]:
doc = nlp(sentences[0])
displacy.render(doc, style="dep")

In [42]:
aspects = []
for sentence in sentences:
    doc = nlp(sentence)
    descriptive_term = ""
    target = ""
    for token in doc:
        if token.dep_ == "nsubj" and token.pos_ == "NOUN":
            target = token.text
        if token.pos_ == "ADJ":
            prepend = ""
            for child in token.children:
                if child.pos_ != "ADV":
                    continue
                prepend += child.text + " "
            descriptive_term = prepend + token.text
    aspects.append({"aspect": target, "description": descriptive_term})
pprint(aspects)

[{'aspect': 'food', 'description': 'delicious'},
 {'aspect': 'time', 'description': 'very enjoyable'},
 {'aspect': 'meal', 'description': 'tasty'},
 {'aspect': 'internet', 'description': 'slow'},
 {'aspect': 'experience', 'description': 'suboptimal'}]


### Classify the sentiment using `TextBlob`

In [43]:
from textblob import TextBlob

for aspect in aspects:
    aspect["sentiment"] = TextBlob(aspect["description"]).sentiment  # or other sentiment classifiers
pprint(aspects)

[{'aspect': 'food',
  'description': 'delicious',
  'sentiment': Sentiment(polarity=1.0, subjectivity=1.0)},
 {'aspect': 'time',
  'description': 'very enjoyable',
  'sentiment': Sentiment(polarity=0.65, subjectivity=0.78)},
 {'aspect': 'meal',
  'description': 'tasty',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)},
 {'aspect': 'internet',
  'description': 'slow',
  'sentiment': Sentiment(polarity=-0.30000000000000004, subjectivity=0.39999999999999997)},
 {'aspect': 'experience',
  'description': 'suboptimal',
  'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}]
